In [52]:
!pip install unidecode
#!python -m spacy download en_core_web_lg

import nltk
import re
import numpy as np
import pandas as pd
import unidecode
import spacy

nltk.download('punkt') # one time execution
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktLanguageVars
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.chunk.regexp import ChunkString, ChunkRule
from nltk.tree import Tree
from nltk.chunk import RegexpChunkParser
from sklearn.metrics.pairwise import cosine_similarity 
from scipy.spatial.distance import cosine
from spacy.lang.en import English


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [124]:
class BulletPointLangVars(PunktLanguageVars):
    sent_end_chars = ('.', ';', ':')

class AnswerRetriever():
    
    stop_words = set(stopwords.words('english'))
    TAG_RE = re.compile(r'<[^>]+>')
            
    def preprocess(self, text):
        #convert to lower case
          text = text.lower()
          
          #remove HTML tags (used for data taken from websites directly)
          #text = TAG_RE.sub('', text)
          
          #remove accents (in case google translate glitches)
          text = unidecode.unidecode(text)
          
          #remove punctuation (easier for posTag)
          text = re.sub(r'[^\w\s]', '', text)
          
          #remove stopwords
          word_tokens = word_tokenize(text)
          filtered_sentence = [w for w in word_tokens if not w in self.stop_words]
          return (' '.join(str(w) for w in filtered_sentence))
    
    def cosine_similarity_calc(self, vec_1,vec_2):
        sim = np.dot(vec_1,vec_2)/(np.linalg.norm(vec_1)*np.linalg.norm(vec_2))
        return sim     

    def join_prefix(self, ar_df):
        prefix = " "
        prefix_index = 0  
        for sentence in ar_df['sentences']: 
            if re.match('^\d+', sentence):
                prefix_index = ar_df.loc[ar_df['sentences'] == sentence].index + 1
                prefix = ar_df.iloc[prefix_index]['sentences']   
            else:
                if (str(prefix) != str(ar_df.loc[ar_df['sentences'] == sentence]['sentences'])): #doesn't work for some reason
                    print(ar_df.loc[prefix_index, 'sentences'])
                    ar_df.at[prefix_index, 'sentences'] = prefix + sentence
                    print(ar_df.loc[prefix_index,'sentences'])
                prefix_index = prefix_index+1
        return ar_df
    

    
    def extractAnswer(self):
        tokenizer = PunktSentenceTokenizer(lang_vars = BulletPointLangVars())
        keyArticle_tokens = tokenizer.tokenize(keyArticle)
        embeddings = spacy.load('en_core_web_lg')
        ar_df = pd.DataFrame(keyArticle_tokens, columns=['sentences'])
        ar_df['question'] = self.preprocess(self.question)
        ar_df['sentences'].apply(self.preprocess)
        ar_df = self.join_prefix(ar_df)
        ar_df['similarity'] = ar_df.apply(lambda x: self.cosine_similarity_calc(embeddings(x['sentences']).vector, embeddings(x['question']).vector), axis=1)       
        ar_df = ar_df.sort_values(['similarity'], ascending=False)
        return ar_df
                                                      
    def __init__(self, question, keyArticle):
        self.question = question
        self.keyArticle = keyArticle

    
        

In [126]:
question = "Can Obvion sell the collateral as a whole?"

keyArticle = '''1. If Obvion proceeds to the forced sale of the collateral, Obvion has the choice
between public sale (via a notary to be designated by it) and private sale
with the approval of the preliminary relief judge of the court.
2. In the event of a forced sale, Obvion is authorized:
a. to subdivide and/or subdivide the collateral into apartment rights;
b. to sell the collateral in parts;
c. sell multiple collateral together;
d. establish auction or sale conditions;
e. to establish easements and/or qualitative obligations;
f. to exercise all rights under the purchase agreement, insofar as the law opposes this
not resist;
g. to postpone or cancel a sale.
3. The mortgagor is obliged:
a. all desired information, data, documents and evidence about the collateral
to be handed over to the notary in charge of the sale. Or to the
preliminary relief judge of the court to whom a request for approval of a
private sale has been submitted.
b. to enable interested parties to view the collateral at a location to be determined by Obvion
set days and times;
c. in the case of public or private sale, the collateral before the commencement of
the auction (public sale) or before the date of the notarial deed of transfer
(private sale) completely and at the free disposal of the buyer
set. The costs of this will be borne by the mortgagor.
4. If Obvion also has a right of pledge on movable property that is intended to hold the collateral
on which the mortgage right is established, Obvion may use the pledged
sell the goods and the collateral together in accordance with the rules that apply to the law of
mortgage.
5. Obvion reduces the debts secured by the right of mortgage with the
proceeds of the forced sale. Obvion is authorized to determine which debts are
the yield are reduced and also the order in which and the extent and manner in which
it happens.
'''

ar = AnswerRetriever(question, keyArticle)
answer_df = ar.extractAnswer()
display(answer_df)


4    a. to subdivide and/or subdivide the collatera...
Name: sentences, dtype: object
4    NaN
Name: sentences, dtype: object
5    b. to sell the collateral in parts;
Name: sentences, dtype: object
5    NaN
Name: sentences, dtype: object
6    c. sell multiple collateral together;
Name: sentences, dtype: object
6    NaN
Name: sentences, dtype: object
7    d. establish auction or sale conditions;
Name: sentences, dtype: object
7    NaN
Name: sentences, dtype: object
8    e. to establish easements and/or qualitative o...
Name: sentences, dtype: object
8    NaN
Name: sentences, dtype: object
9    f. to exercise all rights under the purchase a...
Name: sentences, dtype: object
9    NaN
Name: sentences, dtype: object
10    g. to postpone or cancel a sale.
Name: sentences, dtype: object
10    NaN
Name: sentences, dtype: object
13    a. all desired information, data, documents an...
Name: sentences, dtype: object
13    NaN
Name: sentences, dtype: object
14    Or to the\npreliminary relief judg

TypeError: ignored